<a href="https://colab.research.google.com/github/MatteoPao/cubert_keras/blob/main/colab/NeuronPrediction_HiddenLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuron prediction - Hidden layer

Questo notebook è stato utilizzato per salvare le previsioni dei layer interni del modello, fornendo in input il dataset etichettato per complessità ciclomatica

In [ ]:
!git clone https://github.com/MatteoPao/cubert_keras.git

%cd cubert_keras/src/

In [ ]:
from google.colab import drive                                  
drive.mount('/content/drive/')

#%cp -r /content/drive/MyDrive/Tesi_ModelData/cubert_pretrained_model_exceptions /content/cubert_keras/src/fitness/cubert_fitness/cubert_pretrained_model_exceptions
#%cp -r /content/drive/MyDrive/Tesi_ModelData/cubert_pretrained_model_variablemisuse /content/cubert_keras/src/fitness/cubert_fitness/cubert_pretrained_model_variablemisuse
%cp -r /content/drive/MyDrive/Tesi_ModelData/cubert_pretrained_model_epochs_2 /content/cubert_keras/src/fitness/cubert_fitness/cubert_pretrained_model_epochs_2

In [ ]:
!pip install -r ../requirements.txt

from intermediate_layer import read_examples_from_json
from keras_bert import load_trained_model_from_checkpoint, get_checkpoint_paths
from cubert.full_cubert_tokenizer import FullCuBertTokenizer, CuBertVariableMisuseProcessor, InputExample
from cubert import tokenizer_registry
import progressbar
import keras.backend as K
import keras
import json
import numpy as np
import tensorflow as tf



*   Importazione degli individui dal file   (*read_example_from_json*)
*   Generazione un istanza tokenizer        (*FullCubertTokenizer*)
*   Conversione degli esempi in feature     (*convert_examples_to_features*)



In [ ]:
model_path = "fitness/cubert_fitness/cubert_pretrained_model_variablemisuse"
#model_path = "fitness/cubert_fitness/cubert_pretrained_model_epochs_2"

data_path = "../cyclomatic_complexity/final/Final_1to16_2400_FT/data_cc.jsontxt"
#data_path = "../cyclomatic_complexity/final/Final_1to16_2400_FF/data_cc.jsontxt"

layer_name = "Encoder-24-FeedForward-Norm"

In [ ]:
paths = get_checkpoint_paths(model_path)

examples = read_examples_from_json(data_path, "eval")

#Carica il tokenizer
tokenizer = FullCuBertTokenizer(code_tokenizer_class=tokenizer_registry.TokenizerEnum.PYTHON.value, vocab_file=paths.vocab)
print("Tokenizer loaded")

processor = CuBertVariableMisuseProcessor()
label_list = processor.get_labels()

#Tokenizza tutto il dataset
features = tokenizer.convert_examples_to_features(examples, label_list, 512)
print("Dataset tokenized")

Il seguente blocco di codice rielabora le feature nella corretta forma accettata dal modello.

Nel caso in cui il modello venga importato con il flag *training=True* allora l'input è formato da 3 array [input, segment, mask].

In alternativa con *training=False* l'input è formato solo da [input, segment] 

In [ ]:
inp, seg, mas = [], [], []
label = []

for f in features:
  inp.append(f.input_ids)
  seg.append(f.segment_ids)
  mas.append(f.input_mask)
  label.append(f.label_id)

#with open('../label_VM2800.npy', 'wb') as file:
#   np.save(file, label)

#Nel caso di modello troncato con keras.Model(), utilizzare data_input o data_input_s
"""
data_input = [np.array(inp), np.array(seg), np.array(mas)]
data_input_s = [np.array(inp), np.array(seg)]
"""

#Nel caso di modello troncato con keras.backend l'input deve essere diviso in batch precedentemente
#Prima alternativa, training=True
"""
batched_input = []
for index in range(int(len(inp)/8)):
  step = int(index*8)
  batched_input.append([np.array(inp[step:step+8]), np.array(seg[step:step+8]), np.array(mas[step:step+8])])
"""

#Seconda alternativa, training=False
batched_input = []
for index in range(int(len(inp)/8)):
  step = int(index*8)
  batched_input.append([np.array(inp[step:step+8]), np.array(mas[step:step+8])])
print(np.asarray(batched_input).shape)

In [ ]:
#Carica il checkpoint
model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, training=False)
#model.summary()

#Prima alternativa, tronca il modello con keras.backend (Consigliato per risparmiare memoria)
model = K.function([model.input], [model.get_layer(layer_name).output])

#Seconda alternativa, tronca il modello con keras.Model (Usare solo con dataset di dimensioni ridotte)
# model = keras.Model(inputs=model.input, outputs=model.get_layer(layer).output)

print("Model loaded")

In caso di modello troncato con keras.backend si usa la prima alternativa per la previsione. Quindi l'output viene direttamente salvato su file. Con la seconda alternativa invece l'output è unico (e molto pesante)

In [ ]:
#In caso di modello troncato con keras.backend si usa questo metodo per la previsione
%mkdir results
count = 0
for input in progressbar.progressbar(batched_input):
  batched_output = model([input, 0])[0]
  with open('results/'+ layer_name + '_prediction_' + str(count) + '.npy', 'wb') as file:
    np.save(file, batched_output)
  count += 1

#In caso di modello troncato con keras.Model si usa questo metodo per la previsione
"""
pred_results = model.predict(data_input_s, batch_size=16, verbose=1)
pred_results = np.transpose(pred_results)
"""

Ultimo blocco che carica i file nella cartella drive. Il comando *flush_and_unmount()* permette di salvare su drive le modifiche che sono state appportate durante la sessione colab 

In [ ]:
!zip -r /content/prediction.zip /content/cubert_keras/src/results
%cp -r /content/prediction.zip /content/drive/MyDrive/Tesi_Database/FINAL/prediction_1to16_FT_VM/prediction.zip
#%cp -r /content/prediction.zip /content/drive/MyDrive/Tesi_Database/FINAL/prediction_1to16_FF_VM/prediction.zip
#%cp -r /content/prediction.zip /content/drive/MyDrive/Tesi_Database/FINAL/prediction_1to16_FT_E2/prediction.zip
#%cp -r /content/prediction.zip /content/drive/MyDrive/Tesi_Database/FINAL/prediction_1to16_FF_E2/prediction.zip
drive.flush_and_unmount()